In [1]:
import librosa
import librosa.display

import cv2
import os

import numpy as np
import matplotlib.pyplot as plt

import soundfile as sf
cv2.__version__

'4.5.5'

In [2]:

filename = 'Tears_of_a_witch_SFA.wav'
y,sr = librosa.load(filename,sr=None)
print(y.shape, sr)

(6849536,) 44100


filename = 'cqt.npz'
C = np.load(filename)['spec']
sr = 44100
print(C.shape, sr)

In [3]:
onset_env = librosa.onset.onset_strength(y=y, sr=sr)

# Run the default beat tracker
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
print('Estimated tempo: {:.2f} beats per minute'.format(tempo))

# dynamic tempo
dtempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr,
                            aggregate=None)

beat_times = librosa.frames_to_time(beat_frames, sr=sr)

Estimated tempo: 120.19 beats per minute


In [4]:
print(y.shape[0]/sr)
beat_times[-1]

155.31827664399094


149.8964172335601

In [5]:
# 고조파 타악기 소스 분리(HPSS)
stft = librosa.stft(y)
harmonic, percussive = librosa.decompose.hpss(stft)
y = librosa.istft(harmonic)

In [6]:
print(stft.shape)
print(stft.min(),stft.max())

(1025, 13379)
(-256.57904+53.71582j) (268.1164-19.037182j)


In [7]:
hop_len = 2048
freq_w = 3
octave = 7

C = librosa.cqt(y, sr=sr, hop_length= hop_len, n_bins= 12*freq_w*octave, bins_per_octave= 12*freq_w)

In [8]:
sec = hop_len/sr
print(sec)
print(C.shape)
print(C.min(),C.max())

0.046439909297052155
(252, 3345)
(-15.710788-5.482887j) (15.536943-6.0659804j)


In [9]:
c_src = np.abs(C)
c_max = np.abs(C).max()
c_src = ((c_src+c_max)/(c_max*2)*255).astype(np.uint8)
print(c_src.min(),c_src.max())

127 255


In [10]:
C_db = librosa.amplitude_to_db(np.abs(C),ref=np.max)+80

In [11]:
print(C_db.shape)
print(C_db.min(), C_db.max())

(252, 3345)
0.0 80.0


In [12]:
c_db_src = (C_db/80*255).astype(np.uint8)
print(c_db_src.min(),c_db_src.max())

0 255


In [13]:
gaussian_mask = np.array([[1/16,1/8,1/16], [1/8,1/4,1/8], [1/16,1/8,1/16]])
sharpening_mask1 = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
sharpening_mask2 = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])

In [14]:
def nothing(pos):
    pass

def working():
    cv2.destroyAllWindows()
    raise NotImplementedError    

def trace(func):
    def wrapper():
        print(func.__name__, "시작")
        func()
        print(func.__name__, "끝")
    return wrapper

def imgclip(src):
    result = np.clip(src,0,255).astype(np.uint8)
    return result

@trace
def gaussian(source):
    # gaussian_mask = np.array([[1/16,1/8,1/16], [1/8,1/4,1/8], [1/16,1/8,1/16]])
    # result = cv2.filter2D(source,-1,gaussian_mask)
    result = cv2.GaussianBlur(source, (0,0), 1)
    print("Gaussian filter")
    return result

@trace
def sharpening1(source):
    sharpening_mask1 = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    result = cv2.filter2D(source,-1,sharpening_mask1)
    result = imgclip(result)
    print("8 connected sharpening")
    return result

@trace
def sharpening2(source):
    sharpening_mask2 = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    result = cv2.filter2D(source,-1,sharpening_mask2)
    result = imgclip(result)
    return result

@trace
def stretching(source,pos):
    i_min = pos
    i_max = source.max()
    source = imgclip(source.astype(np.int16)-i_min)
    result = (255/(i_max-i_min)*(source))
    result = imgclip(result)
    print("{}~{} stretcing".format(i_min,i_max))
    return result

@trace
def n_under0(source, pos):
    source[source<=pos]=0
    print("%d under 0"%pos)
    return source

@trace
def div(source, pos):
    source = (source*(pos/100))
    source = imgclip(source)
    print("%.2f div_fn"%(pos/100))
    return source

@trace
def value_up(source, pos):
    h,w = source.shape[:2]
    a = np.ones((h,w),np.uint8)*pos
    source = cv2.add(source,a)
    print("up shift %d"%pos)
    return source
    
@trace
def erosion(src):
    kernel = np.ones((3,3),np.uint8)
    result = cv2.erode(src,kernel)
    return result

@trace
def dilation(src):
    kernel = np.ones((3,3),np.uint8)
    result = cv2.dilate(src,kernel)
    return result

@trace
def equalize(src):
    result = cv2.equalizeHist(src)
    return result


def fn_log_write(log,fn,pos=None):
    if pos is None:
        log.append(fn)
    else:
        log.append((fn,pos))
    return log

def auto_fn(src,log):
    for kw in log:
        if type(kw) is tuple:
            src = kw_fn(kw[0],src,kw[1])
        else:
            src = kw_fn(kw,src)            
    return src


def kw_fn(kw,src,pos=None,log=None):
    if kw==ord('0'):
        src = n_under0(src,pos)
    elif kw==ord('t'):
        src = stretching(src,pos)
    elif kw==ord('g'):
        src = gaussian(src)
    elif kw==ord('1'):
        src = erosion(src)
    elif kw==ord('2'):
        src = dilation(src)
    elif kw==ord('d'):
        src = div(src,pos)
    elif kw==ord('u'):
        src = value_up(src,pos)
    elif kw==ord('e'):
        src = equalize(src)
        
    if log is None:
        return src
    else:
        log = fn_log_write(log,kw,pos)
        return src, log
    
def iskw(kw):
    kw_fn_list =[
        ord('0'),ord('t'),ord('g'),ord('1'),ord('2'),ord('d'),ord('u'),ord('e')
    ]
    if kw in kw_fn_list:
        return True
    else:
        return False
        
def jet(gray_img):
    r,c = gray_img.shape
    img_hsv = np.full((r,c,3),(130,255,255),dtype=np.uint8)
    hue = img_hsv[:,:,0]
    hue -= (gray_img.copy()/2).astype(np.uint8)
    img_bgr = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR_FULL)
    return img_bgr

def getGrayHistImage(hist):
    imgHist = np.full((200, 256), 255, dtype=np.uint8)

    histMax = np.max(hist)
    if histMax==0:
        histMax=1
    for x in range(256):
        pt1 = (x, 200)
        pt2 = (x, 200 - int(hist[x, 0] * 200 / histMax))
        cv2.line(imgHist, pt1, pt2, 0)

    return imgHist

def mk_sec_idxlist(src,sec,sr=44100,hop_length=512,):
    fps = sr/hop_length
    idxlist = []
    frame = int(sec*fps)
    _, length = src.shape
    if length < frame:
        return [(0,length)]
    else:
        idx = 0
        while True:
            if frame*(idx+1) > length:
                idxlist.append((frame*idx,length))
                break
            else:
                idxlist.append((frame*idx,frame*(idx+1)))
                idx += 1
            
        return idxlist

def cqt_load(src,idx):
    print(idx[0],idx[1])
    result = src[:,idx[0]:idx[1]].copy()
    return result

# def mkmask(src,idx,sec):
    

In [ ]:
def mouse_eraser(event, x,y, flags, params):
    global crop
    pos = cv2.getTrackbarPos('base_level','spectrum')
    
    if event==cv2.EVENT_MOUSEMOVE:
        canvas = cv2.flip(crop, 0)
        cv2.circle(canvas,(x,y),pos//25,255,2,cv2.LINE_AA)
        img_bgr = jet(canvas)
        cv2.imshow("spectrum", img_bgr)
        
    if flags & cv2.EVENT_FLAG_LBUTTON:
        canvas = cv2.flip(crop, 0)
        cv2.circle(canvas,(x,y),pos//25,0,-1,cv2.LINE_AA)
        img_bgr = jet(canvas)
        cv2.imshow("spectrum", img_bgr)
        crop = cv2.flip(canvas, 0)

src = c_src.copy()

cv2.namedWindow('spectrum', cv2.WINDOW_NORMAL)
cv2.createTrackbar('base_level','spectrum',0,255,nothing)
cv2.setMouseCallback('spectrum', mouse_eraser)

global crop

fn_log = []
idx = 0
sec = 10
magin = 0

idxlist = mk_sec_idxlist(src,sec,sr=sr,hop_length=hop_len)
crop = cqt_load(src,idxlist[idx])

while True:
    hist = cv2.calcHist([crop], [0], None, [256], [1, 257])
    histImg = getGrayHistImage(hist)
    cv2.imshow('histogram', histImg)
    
    img_bgr = cv2.flip(jet(crop),0)
    cv2.imshow('spectrum', img_bgr)
        
    kw=cv2.waitKey()
    pos = cv2.getTrackbarPos('base_level','spectrum')
    
    if kw==27:
        break
    elif kw==ord('r'):
        crop = cqt_load(src,idxlist[idx])
        fn_log.clear()
        print()
        print('-'*10)
        print("Reset")
        print('-'*10)
        print()
    elif kw==ord('z'):
        if fn_log == []:
            print("nothing log")
        else:
            fn_log.pop()
            crop = cqt_load(src,idxlist[idx])
            crop = auto_fn(crop,fn_log)
    elif iskw(kw):
        crop, fn_log = kw_fn(kw,crop,pos,fn_log)
    elif kw==ord('a'):
        print("auto repeat start")
        if fn_log==[]:
            print('nothing log')
        else:
            crop = auto_fn(crop,fn_log)
        print("auto repeat end")
    elif kw==ord('n'):
        idx +=1
        if idx >= len(idxlist):
            idx -=1
            print('end index')
        crop = cqt_load(src,idxlist[idx])
    elif kw==ord('b'):
        idx -=1
        if idx < 0:
            idx = 0
            print('start index')
        crop = cqt_load(src,idxlist[idx])
    elif kw==ord('s'):
        beat_time_list = beat_times[(sec*idx < beat_times)&(beat_times < sec*(idx+1))] - sec*idx
        notes = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']
        y_ticks = np.arange(0,12*freq_w*octave,freq_w)
        y_list=[]

        for o in range(octave):
            for i in range(12):
                y_list.append(notes[i]+'%d'%(o+1))

        plt.style.use('default')
        fig, ax = plt.subplots(1,1,figsize=(15,10))
        img = librosa.display.specshow(crop, sr=sr, x_axis='time', ax=ax, hop_length= hop_len, bins_per_octave= 12*freq_w,
                                    cmap='jet'
                                    )
        ax.set(title=filename+str(idx))
        ax.set_yticks(y_ticks)
        ax.set_yticklabels(y_list)
        
        for i,v in zip(range(len(beat_time_list)),beat_time_list):
            if i % 4==0:
                ax.axvline(v,c='white')
            else:
                ax.axvline(v,c='k')
                
        for i in np.arange(0,12*freq_w*octave,freq_w):
            if i % (freq_w*12) ==0:
                ax.axhline(i,c='g')
            elif i % (freq_w*2) ==0:
                ax.axhline(i,c='k')
            else:
                ax.axhline(i,c='white')
            
        plt.savefig('./output/'+filename.split('.')[0]+str(idx))
        print(filename.split('.')[0]+str(idx),'save complete')
        plt.show()
        
        src[:,idxlist[idx][0]:idxlist[idx][1]] = crop.copy()
    
    else:
        continue
            

cv2.destroyAllWindows();

In [16]:
# c_signed = np.where(C!=0,C/np.abs(C),0.)
_, mask = cv2.threshold(src,50,1, cv2.THRESH_BINARY)
print(mask.shape)
print(mask.min(),mask.max())

(252, 3345)
1 1


In [17]:
icqt_src = np.where(mask,C,0.)
print(icqt_src.shape)
print(icqt_src.min(),icqt_src.max())

(252, 3345)
(-15.710788-5.482887j) (15.536943-6.0659804j)


In [18]:
y = librosa.icqt(C= icqt_src, sr= sr, hop_length= hop_len, bins_per_octave= freq_w*12)
sf.write('./output/icqt.wav',y,sr,"PCM_24")